In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
train = pd.read_csv('train.csv')
train.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [ ]:
#PreProcessing
train= train.replace(["female" , "male"] , [0 , 1])
train = train.replace(["C" , "S" , "Q"] , [0 , 1 , 2])
train = train.fillna(0)
train.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,0,1.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,0.0


In [2]:
Y_train =  np.array(train[['Survived']])
X_train = np.array(train[['Pclass', 'Sex', 'Fare', 'Age', 'SibSp', 'Parch', 'Embarked']])
print(X_train.shape , Y_train.shape)

NameError: name 'train' is not defined

In [ ]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(7,activation='sigmoid'),
                                    tf.keras.layers.Dense(8,activation='relu'),
                                    tf.keras.layers.Dense(6,activation='relu'),
                                    tf.keras.layers.Dense(4,activation='relu'),
                                    tf.keras.layers.Dense(2,activation='softmax')
])

NameError: name 'tf' is not defined

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

In [ ]:
output=model.fit(X_train,Y_train,epochs=500)

In [ ]:
plt.plot(output.history['loss'], c = 'red')
plt.plot(output.history['accuracy'], c ='green')
plt.xlabel('Epoch')
plt.ylabel('Loss')

In [ ]:
test = pd.read_csv('drive/MyDrive/Colab Notebooks/Titanic Dataset/test.csv')
test = test.replace(['female','male'],[0, 1])
test = test.replace(['S','C','Q'],[0, 1, 2])
test=test.fillna(0)
test.head()

In [ ]:
survived = pd.read_csv('gender_submission.csv')
survived.head(2)

In [ ]:
X_test = np.array(test[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']])
Y_test = np.array(survived[['Survived']])
print(X_train.shape , Y_train.shape)


In [ ]:
from keras import layers
layer = layers.Normalization()
layer.adapt(X_test)
X_test = layer(X_test).numpy()

In [ ]:
acc=model.evaluate(X_test,Y_test)

In [ ]:
Kelly=np.array([3,1,27,0,0,8.6,0])
Kelly=Kelly.reshape(1,7)
y_pred=model.predict(Kelly)
prediction=np.argmax(y_pred)
print('prediction  ----> ',prediction)

In [ ]:
#___________KNN_____________#
class kNearestNeighbors(): 


    def __init__(self, k):
        self.k = k 


    def fit(self, X_train , y_train): 
        self.X_train = X_train
        self.y_train = y_train
        self.number_classes = len(np.unique(y_train))


    def nearNeighbors(self , X_test):
        dist = np.sqrt(np.sum((X_test - self.X_train)**2 ,axis=1))
        near_neighbors = np.argsort(dist)[:self.k]
        return near_neighbors

        
    def predict (self, X_test):
        near_neighbors = self.nearNeighbors(X_test)
        y = np.argmax(np.bincount(self.y_train[near_neighbors]))
        return y

    def Evaluate (self, X_test, y_test):
        Succes=0
        Test_len = len(X_test)
        for i in range (Test_len):
            Prediction = self.predict(X_test[i])
            if Prediction == y_test[i]:
                Succes = Succes+1 
        Accuracy = Succes / Test_len
        return Accuracy

In [ ]:
Y_train = Y_train.reshape(891,)
KNN = kNearestNeighbors(k=5)
KNN.fit(X_train,Y_train)
y = []
for xts in X_test:
    y.append(KNN.predict(xts))

print(' Accuracy of ___________KNN_____________ is', KNN.Evaluate(X_test, Y_test))

In [ ]:
#___________Adaline_____________#

from numpy.linalg import inv
class AdalineClassifi(): 


    def __init__(self):
        pass

    def fit(self, x_train, y_train):
        # W = (X.T * X)^-1 * (X.T * Y)
        self.w=np.matmul(inv(np.matmul(x_train.T,x_train)),np.matmul(x_train.T,y_train))
        return self.w
        
    def predict(self, x_test):
        y_predict = np.matmul(x_test, self.w)
        y=[]
        for i in range(len(y_predict)):
            if y_predict[i]<0.5:
                y.append(0)
            else:
                y.append(1)  
        y=np.array(y)
        return y
    
    def Evaluate (self, X_test, Y_test):
        pred = self.predict(X_test)
        Succes=0
        for i in range (len(X_test)):
           if pred[i] == Y_test[i]:
               Succes = Succes+1 
           Accuracy = Succes / len(X_test)
        return Accuracy

In [ ]:
model = AdalineClassifi()
model.fit(X_train,Y_train)
print(model.predict(X_test))
print(' Accuracy of ___________Adaline_____________ is', model.Evaluate(X_test, Y_test))

In [ ]:
#___________Perceptron_____________

class perceptron:
    def __init__ (self):
        pass

    def fit(self, X, Y  , epochs):

        lr = 0.000001
        self.w = np.random.rand(7)
        self.b = np.random.rand(1)
        self.Error = []
        for i in range (epochs):
            for i in range (X.shape[0]):
                y_pred = np.matmul(X[i], self.w ) + self.b
                e = Y[i] - y_pred
                self.w += lr * X[i] * e
                self.b += lr * e 

                Y_pred = np.matmul(X, self.w) + self.b
                error = np.mean(np.abs(Y - Y_pred))
                self.Error.append(error)
        np.save('w', self.w)
        np.save('b', self.b)
        return self.Error

    def predict (self, X):
        w = np.load('w.npy')
        b = np.load('b.npy')
        Y_pred=np.matmul(X, w) + b
        return  Y_pred

    def evaluate(self, X, Y):
        w = np.load('w.npy')
        b = np.load('b.npy')
        Y_pred = np.matmul(X, w) + b
        evaluation = np.mean(np.abs(Y - Y_pred))
        return evaluation

In [ ]:
model = perceptron()
model.fit(X_train, Y_train ,2)
model.predict(X_test)
model.evaluate(X_test, Y_test)
print('Accuracy of ___________Perceptron_____________ is:', model.evaluate(X_test, Y_test) )